## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,2022-04-09 18:33:18,62.4560,-114.3525,25.41,68,20,20.71,few clouds
1,1,Willmar,US,2022-04-09 18:31:55,45.1219,-95.0433,50.02,44,0,5.75,clear sky
2,2,Ushuaia,AR,2022-04-09 18:33:19,-54.8000,-68.3000,51.42,57,20,0.00,few clouds
3,3,Amalapuram,IN,2022-04-09 18:33:20,16.5833,82.0167,79.93,85,87,3.02,overcast clouds
4,4,Mataura,NZ,2022-04-09 18:33:20,-46.1927,168.8643,42.44,98,69,2.53,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Amalapuram,IN,2022-04-09 18:33:20,16.5833,82.0167,79.93,85,87,3.02,overcast clouds
10,10,Rikitea,PF,2022-04-09 18:33:24,-23.1203,-134.9692,79.07,73,6,18.34,clear sky
11,11,Namatanai,PG,2022-04-09 18:33:24,-3.6667,152.4333,78.40,80,100,2.19,overcast clouds
12,12,Castro,BR,2022-04-09 18:33:25,-24.7911,-50.0119,77.41,61,82,6.38,broken clouds
16,16,Lorengau,PG,2022-04-09 18:33:26,-2.0226,147.2712,79.92,82,100,6.13,overcast clouds
24,24,Kapaa,US,2022-04-09 18:33:31,22.0752,-159.3190,77.18,80,100,11.50,overcast clouds
26,26,Vaini,TO,2022-04-09 18:33:31,-21.2000,-175.2000,77.16,88,75,10.36,broken clouds
28,28,Arraial Do Cabo,BR,2022-04-09 18:33:33,-22.9661,-42.0278,84.16,78,0,12.66,clear sky
30,30,Port Blair,IN,2022-04-09 18:33:34,11.6667,92.7500,78.84,89,20,3.44,mist
33,33,Saint George,US,2022-04-09 18:32:32,37.1041,-113.5841,80.64,13,0,3.44,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                229
City                   229
Country                229
Date                   229
Lat                    229
Lng                    229
Max Temp               229
Humidity               229
Cloudiness             229
Wind Speed             229
Current Description    229
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Amalapuram,IN,79.93,overcast clouds,16.5833,82.0167,
10,Rikitea,PF,79.07,clear sky,-23.1203,-134.9692,
11,Namatanai,PG,78.40,overcast clouds,-3.6667,152.4333,
12,Castro,BR,77.41,broken clouds,-24.7911,-50.0119,
16,Lorengau,PG,79.92,overcast clouds,-2.0226,147.2712,
24,Kapaa,US,77.18,overcast clouds,22.0752,-159.3190,
26,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,
28,Arraial Do Cabo,BR,84.16,clear sky,-22.9661,-42.0278,
30,Port Blair,IN,78.84,mist,11.6667,92.7500,
33,Saint George,US,80.64,clear sky,37.1041,-113.5841,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/directions/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")   
        

KeyError: 'results'

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
